# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="c:/Tennis_Windows_x86_64/Tennis.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [5]:
from agent import Agent
import torch
import pickle as pkl

In [6]:
agent1=Agent(action_size=action_size,state_size=state_size,random_seed=42)
agent2=Agent(action_size=action_size,state_size=state_size,random_seed=42)

In [7]:
from collections import deque
scores_mean = deque(maxlen=100)
scores = []
done = False
x = 0
eps = 1
while not done:
    x += 1
    env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
    state = env_info.vector_observations                  # get the current state (for each agent)
    score = np.zeros(2)
    agent1.reset()
    agent2.reset()
    eps = np.max([eps*0.9998,0.1])
    noise = np.random.random() < eps
    while True:
        action=[agent1.act(state[0],add_noise=noise),agent2.act(state[1],add_noise=noise)]
        env_info = env.step(action)[brain_name]
        next_state = env_info.vector_observations
        reward = env_info.rewards
        dones = env_info.local_done                        # see if episode finished
        score += env_info.rewards                         # update the score (for each agent)
        agent1.step(actions=action[0],dones=dones[0],next_states=next_state[0],rewards=reward[0],states=state[0])
        agent2.step(actions=action[1],dones=dones[1],next_states=next_state[1],rewards=reward[1],states=state[1])
        
        state = next_state # roll over states to next time step
        if np.any(dones):
            scores_mean.append(score.max())
            scores.append(score.max())
            if np.mean(scores_mean) > 0.5:
                done = True
            print("Episode {:>4d}: \tscore {:.3f}\tmean {:.3f}\tepsilon {:.3f}                         ".format(x,score.max(),np.mean(scores_mean),eps),end='\r')
            if x % 100 == 0:
                q0,q1,q2,q3,q4 =np.percentile(scores_mean, [0,25, 50, 75,100])
                print("Episode {:>4d}: \tmean {:.3f} \tmin {:.3f}\tQ1 {:.3f}\tmedian {:.3f}\tQ3 {:.3f}\tmax {:.3f}\tsd {:.3f}".format(x,np.mean(scores_mean),q0,q1,q2,q3,q4,np.std(scores_mean)))
                torch.save(agent1.actor_local.state_dict(), "checkpoints/actor1.episode{}.pth".format(x))
                torch.save(agent2.actor_local.state_dict(), "checkpoints/actor2.episode{}.pth".format(x))
                torch.save(agent1.critic_local.state_dict(), "checkpoints/critic1.episode{}.pth".format(x))
                torch.save(agent2.critic_local.state_dict(), "checkpoints/critic2.episode{}.pth".format(x))
            break
q0,q1,q2,q3,q4 = np.percentile(scores_mean, [0,25, 50, 75,100])
print("Solved in {} episodes: mean score {:.3f}\tmin {:.3f}\tQ1 {:.3f}\tmedian {:.3f}\tQ3 {:.3f}\tmax {:.3f}                              ".format(x,np.mean(scores_mean),q0,q1,q2,q3,q4))
torch.save(agent1.actor_local.state_dict(), "actor1.final.pth")
torch.save(agent2.actor_local.state_dict(), "actor2.final.pth")
torch.save(agent1.critic_local.state_dict(), "critic1.final.pth")
torch.save(agent2.critic_local.state_dict(), "critic2.final.pth")
with open("scores.pkl","wb") as f:
    pkl.dump(file=f,obj=scores)

Episode  100: 	mean 0.005 	min 0.000	Q1 0.000	median 0.000	Q3 0.000	max 0.100	sd 0.021
Episode  200: 	mean 0.000 	min 0.000	Q1 0.000	median 0.000	Q3 0.000	max 0.000	sd 0.000
Episode  300: 	mean 0.005 	min 0.000	Q1 0.000	median 0.000	Q3 0.000	max 0.100	sd 0.022
Episode  400: 	mean 0.014 	min 0.000	Q1 0.000	median 0.000	Q3 0.000	max 0.100	sd 0.035
Episode  500: 	mean 0.003 	min 0.000	Q1 0.000	median 0.000	Q3 0.000	max 0.100	sd 0.017
Episode  600: 	mean 0.012 	min 0.000	Q1 0.000	median 0.000	Q3 0.000	max 0.100	sd 0.032
Episode  700: 	mean 0.035 	min 0.000	Q1 0.000	median 0.000	Q3 0.100	max 0.100	sd 0.048
Episode  800: 	mean 0.031 	min 0.000	Q1 0.000	median 0.000	Q3 0.100	max 0.100	sd 0.046
Episode  900: 	mean 0.036 	min 0.000	Q1 0.000	median 0.000	Q3 0.090	max 0.190	sd 0.048
Episode 1000: 	mean 0.056 	min 0.000	Q1 0.000	median 0.090	Q3 0.100	max 0.190	sd 0.053
Episode 1100: 	mean 0.061 	min 0.000	Q1 0.000	median 0.100	Q3 0.100	max 0.300	sd 0.058
Episode 1200: 	mean 0.083 	min 0.000	Q1 0.0

In [8]:
env.close()